In [1]:
import shutil
import urllib.request as request
from contextlib import closing
import zipfile
import shutil
import sys
import os
from pathlib import Path
import pandas as pd
from simpledbf import Dbf5
import platform

def download(url, save_filepath):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(save_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)

def download_ftp(url, savepath):
    with closing(request.urlopen(url)) as r:
        with open(savepath, 'wb') as f:
            shutil.copyfileobj(r, f)

root = Path("C://Users/Paulo Augusto/Documents/datasets/")

In [2]:
dbc_save_folder = root / "SINASC_DBC"
dbf_save_folder = root / "SINASC_DBF"
csv_save_folder = root / "SINASC_CSV"

dbc_save_folder.mkdir(exist_ok=True)
dbf_save_folder.mkdir(exist_ok=True)
csv_save_folder.mkdir(exist_ok=True)


In [3]:
SINASC_URLS = [
    f"ftp://ftp.datasus.gov.br/dissemin/publicos/SINASC/1996_/Dados/DNRES/DNSP{year}.dbc"
    for year in range(2010, 2019 + 1)
]



for url in SINASC_URLS:
    print("Downloading", url.split("/")[-1])
    save_path = dbc_save_folder / url.split("/")[-1]
    download_ftp(url, save_path)

In [4]:
url = "ftp://ftp.datasus.gov.br/tabwin/tabwin/TAB415.zip"
save_path = root / url.split("/")[-1]
download_ftp(url, save_path)

extract_folder = save_path.parent / save_path.name.replace(".zip", "")
extract_folder.mkdir(exist_ok=True)

with zipfile.ZipFile(save_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

dec_tool_path = extract_folder / "dbf2dbc.exe"

In [5]:
dec_tool_path

WindowsPath('C:/Users/Paulo Augusto/Documents/datasets/TAB415/dbf2dbc.exe')

In [6]:
args = [
    dec_tool_path,
    dbc_save_folder / "*.dbc",
    dbf_save_folder,
]
if not (platform.system() == "Windows"):
    args = ["wine"] + args

[WindowsPath('C:/Users/Paulo Augusto/Documents/datasets/TAB415/dbf2dbc.exe'),
 WindowsPath('C:/Users/Paulo Augusto/Documents/datasets/SINASC_DBC/*.dbc'),
 WindowsPath('C:/Users/Paulo Augusto/Documents/datasets/SINASC_DBF')]

In [7]:
from subprocess import Popen, PIPE

p = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
output, err = p.communicate()
rc = p.returncode

In [8]:
print(output.decode())

DATASUS dbf2dbc 1.0: Comprime arquivos DBF e Expande arquivos DBC
-----------------------------------------------------------------------
                regs       bytes arquivo
-----------------------------------------------------------------------
DBC ==> DBF   598473    17668847 DNSP2009.dbc...  93.84% OK
DBC ==> DBF   601352    20076605 DNSP2010.DBC... -41.78% OK
DBC ==> DBF   610222    24782699 DNSP2011.DBC...   8.17% OK
DBC ==> DBF   616608    36660643 DNSP2012.DBC...  12.48% OK
DBC ==> DBF   610896    37016540 DNSP2013.dbc...  54.23% OK
DBC ==> DBF   625687    44313801 DNSP2014.dbc...  -3.03% OK
DBC ==> DBF   634026    42640365 DNSP2015.dbc...   7.54% OK
DBC ==> DBF   601437    40690695 DNSP2016.dbc... -35.90% OK
DBC ==> DBF   611803    42947030 DNSP2017.dbc... -20.81% OK
DBC ==> DBF   606146    29673856 DNSP2018.DBC...  23.35% OK
DBC ==> DBF   583191    28609376 DNSP2019.dbc...   6.25% OK



In [9]:

for dbf_filepath in dbf_save_folder.glob("*.dbf"):
    print(dbf_filepath)
    csv_filepath = csv_save_folder / dbf_filepath.name.replace(".dbf", ".csv")
    dbf = Dbf5(dbf_filepath)
    dbf.to_csv(csv_filepath)
    

C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2009.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2010.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2011.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2012.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2013.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2014.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2015.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2016.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2017.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2018.dbf
C:\Users\Paulo Augusto\Documents\datasets\SINASC_DBF\DNSP2019.dbf


In [10]:
union_df = None
dfs = []
cols = ["DTNASC", "QTDFILVIVO", "SEXO", "IDADEMAE", "PESO", "GRAVIDEZ", "CONSULTAS", "RACACOR", "CODMUNNASC", "ESTCIVMAE", "ESCMAE", "PARTO", "IDANOMAL"]
union_csv_filename = "union.csv"
for csv_filepath in csv_save_folder.glob("*.csv"):
    if csv_filepath.name == union_csv_filename:
        continue
    df = pd.read_csv(csv_filepath, usecols=cols)
    print(csv_filepath, len(df.columns))
    sorted_cols = list(sorted(df.columns))
    df = df[cols]
    dfs += [df]
    if union_df is None:
        union_df = df
    else:
        union_df = pd.concat([union_df, df])
union_df.to_csv(csv_save_folder / union_csv_filename)

C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2009.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2010.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2011.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2012.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2013.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2014.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2015.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2016.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2017.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2018.csv 13
C:\Users\Paulo Augusto\Documents\datasets\SINASC_CSV\DNSP2019.csv 13


In [11]:
# cols = ["DTNASC", "QTDFILVIVO", "SEXO", "IDADEMAE", "PESO", "GRAVIDEZ", "CONSULTAS", "RACACOR", "CODMUNNASC", "ESTCIVMAE", "ESCMAE", "PARTO", "IDANOMAL"]
